In [19]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import torch
import os, json, cv2, random
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
import detectron2.data.transforms as T


In [14]:
# https://github.com/facebookresearch/detectron2/blob/main/configs/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model=DefaultPredictor(cfg)

In [13]:
from detectron2.modeling import build_model
model = build_model(cfg)
from detectron2.checkpoint import DetectionCheckpointer
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)  # load a file, usually from cfg.MODEL.WEIGHTS


NameError: name 'cfg' is not defined

In [65]:
class Inference:
    def __init__(self, cfg):
        self.cfg = cfg
        self.model = build_model(cfg)
        DetectionCheckpointer(self.model).load(cfg.MODEL.WEIGHTS)  # Load model weights
        self.model.eval()  # Set the model to evaluation mode
        self.model = torch.compile(self.model)

    def predict(self, img):
        original_image = img
        input_format = self.cfg.INPUT.FORMAT
        aug = T.ResizeShortestEdge(
            [self.cfg.INPUT.MIN_SIZE_TEST, self.cfg.INPUT.MIN_SIZE_TEST], self.cfg.INPUT.MAX_SIZE_TEST
        )
        with torch.no_grad():
            # Apply pre-processing to image.
            if input_format == "RGB":
                original_image = original_image[:, :, ::-1]
            height, width = original_image.shape[:2]
            image = aug.get_transform(original_image).apply_image(original_image)
            image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
            image = image.to(self.cfg.MODEL.DEVICE)

            inputs = {"image": image, "height": height, "width": width}
            predictions = self.model([inputs])[0]
        
        return predictions



In [8]:
# Example usage:
output_path= "output/output_test_1.jpg"
inference_instance = Inference(cfg)
img = cv2.imread("input/input.jpg")
predictions, original_image = inference_instance.predict(img)
v = Visualizer(original_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(predictions["instances"].to("cpu"))
cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

W1007 13:43:01.062000 140463490412544 torch/fx/experimental/symbolic_shapes.py:4449] [8/0_1] xindex is not in var_ranges, defaulting to unknown range.
W1007 13:43:01.111000 140463490412544 torch/fx/experimental/symbolic_shapes.py:4449] [8/0_1] xindex is not in var_ranges, defaulting to unknown range.
W1007 13:43:01.155000 140463490412544 torch/fx/experimental/symbolic_shapes.py:4449] [8/0_1] xindex is not in var_ranges, defaulting to unknown range.
W1007 13:43:01.199000 140463490412544 torch/fx/experimental/symbolic_shapes.py:4449] [8/0_1] xindex is not in var_ranges, defaulting to unknown range.
W1007 13:43:01.243000 140463490412544 torch/fx/experimental/symbolic_shapes.py:4449] [8/0_1] xindex is not in var_ranges, defaulting to unknown range.
/home/ubuntu/segmentation/venv/lib/python3.10/site-packages/torchvision/_meta_registrations.py:173: FutureWarning: `create_unbacked_symint` is deprecated, please use `new_dynamic_size` instead
  num_to_keep = ctx.create_unbacked_symint()
/home/u

True

In [9]:
img = cv2.imread("input/input.jpg")
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(predictions["instances"].to("cpu"))
output_path="./output/output_img2.jpg"
print(predictions['instances'])
cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

Instances(num_instances=12, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([[253.5919, 166.8629, 337.8610, 410.6305],
        [113.9906, 267.3970, 150.8854, 397.5742],
        [ 49.9463, 274.8264,  80.3432, 347.1399],
        [386.3561, 272.1247, 412.8921, 302.1220],
        [561.4421, 271.2292, 597.4097, 380.3239],
        [  1.7784, 281.9350,  76.7477, 478.0841],
        [522.6229, 280.9528, 562.6261, 383.9077],
        [404.0339, 273.3655, 461.3947, 347.6321],
        [353.9372, 269.4009, 385.4023, 298.6614],
        [341.6026, 268.6268, 363.2370, 299.0686],
        [593.4126, 268.4496, 614.2268, 314.5462],
        [509.0144, 286.8127, 534.0659, 338.3869]], device='cuda:0')), scores: tensor([0.9973, 0.9967, 0.9888, 0.9843, 0.9800, 0.9797, 0.9702, 0.9545, 0.9208,
        0.7543, 0.5560, 0.5194], device='cuda:0'), pred_classes: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), pred_keypoints: tensor([[[2.9393e+02, 1.9352e+02, 1.3938e+00],
         [2.

True

In [31]:
predictions.keys()
Instances=predictions['instances']
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(predictions["instances"][:1].to("cpu"))
output_path="./output/firstinstances.jpg"
print(predictions['instances'][:1])
cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

Instances(num_instances=1, image_height=480, image_width=640, fields=[pred_boxes: Boxes(tensor([[253.5919, 166.8629, 337.8610, 410.6305]], device='cuda:0')), scores: tensor([0.9973], device='cuda:0'), pred_classes: tensor([0], device='cuda:0'), pred_keypoints: tensor([[[2.9393e+02, 1.9352e+02, 1.3938e+00],
         [2.9991e+02, 1.8932e+02, 1.0178e+00],
         [2.9393e+02, 1.8932e+02, 1.4148e+00],
         [3.1485e+02, 1.9411e+02, 7.1174e-01],
         [3.1485e+02, 1.9411e+02, 1.7536e-01],
         [3.2143e+02, 2.2706e+02, 1.7043e-01],
         [2.9154e+02, 2.2286e+02, 2.2717e-01],
         [3.0589e+02, 2.6299e+02, 4.1604e-01],
         [2.8497e+02, 2.5521e+02, 1.0498e-01],
         [2.7600e+02, 2.6838e+02, 2.9833e-01],
         [2.7182e+02, 2.6778e+02, 1.0078e-01],
         [3.1366e+02, 2.8815e+02, 5.9619e-02],
         [2.9095e+02, 2.8336e+02, 7.8773e-02],
         [3.0170e+02, 3.1989e+02, 5.3951e-02],
         [2.6525e+02, 3.1091e+02, 3.3109e-02],
         [3.1186e+02, 3.8038e+02, 

True

## Panoptic 

In [68]:
#  https://github.com/facebookresearch/detectron2/blob/main/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml 
cfg_Panoptic = get_cfg()
cfg_Panoptic.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg_Panoptic.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg_Panoptic.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
cfg_Panoptic.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PanopticInstance = Inference(cfg_Panoptic)

In [71]:
img = cv2.imread("input/input.jpg")
print(len(PanopticInstance.predict(img)))
panoptic_seg,seg_info=PanopticInstance.predict(img)["panoptic_seg"]
# v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
# out = v.draw_instance_predictions(predictions["instances"].to("cpu"))
# output_path="./output/output_panoptic.jpg"
# print(predictions['instances'])
# cv2.imwrite(output_path, out.get_image()[:, :, ::-1])
# Load the image
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg_Panoptic.DATASETS.TRAIN[0]), scale=1.2)

# Visualize panoptic segmentation
print("shape panoptic seg",panoptic_seg.shape)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), seg_info)

# Show or save the result
# result_image = out.get_image()[:, :, ::-1]
output_path="./output/output_panoptic.jpg"
cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

3
shape panoptic seg torch.Size([480, 640])


True